<a href="https://colab.research.google.com/github/alex-jk/SWB-GVCEH/blob/main/models/relevance_model/Reddit_Relevancy_Models_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install setfit
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
#drive.mount('/content/drive')

import gdown
import json

In [ ]:
# https://drive.google.com/file/d/15reL84i-9niSW4Oa0_1oxR82-5UpAybo/view?usp=sharing

url = 'https://drive.google.com/uc?id=15reL84i-9niSW4Oa0_1oxR82-5UpAybo'
output_reddit_data = 'Complete_Data_v3.json'  # Replace with the actual file name and extension
gdown.download(url, output_reddit_data, quiet=False)

with open(output_reddit_data, 'r') as f:
    reddit_data = json.load(f)

reddit_data_df = pd.DataFrame(reddit_data)
print(f"\ncd_test shape: {reddit_data.shape}")
print(reddit_data.columns)

print("\n------------------------------")
print(reddit_data.head())